## Samantha Beard

#### Term Project Milestone 3: Cleaning/Formatting Website Data

### Setup

In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import codecs
import os

In [2]:
# import website
url = 'https://en.wikipedia.org/wiki/Grammy_Awards'
page = requests.get(url)
grammy_soup = BeautifulSoup(page.text, 'html')

In [3]:
# find table that I am wanting to use
grammy_edition_table = grammy_soup.find_all('table', class_ = 'wikitable')[0]

In [4]:
# functions to parse the wikitable

# this function will determine how many rows and columns should be in the table to prevent data from being empty
def pre_process_table(grammy_edition_table):
    # <tr> = table row
    rows = [x for x in grammy_edition_table.find_all('tr')]

    num_rows = len(rows)

    # get an initial column count. Most often, this will be accurate
    # <th> = table header and <td> = table data
    num_cols = max([len(x.find_all(['th','td'])) for x in rows])

    # I did not see column spans, however, to ensure that all the data is returned we will check
    header_rows_set = [x.find_all(['th', 'td']) for x in rows if len(x.find_all(['th', 'td']))>num_cols/2]

    num_cols_set = []

    for header_rows in header_rows_set:
        num_cols = 0
        for cell in header_rows:
            row_span, col_span = get_spans(cell)
            num_cols+=len([cell.getText()]*col_span)

        num_cols_set.append(num_cols)

    num_cols = max(num_cols_set)

    return (rows, num_rows, num_cols)                          
    

# function to get which rows and columns have spans
def get_spans(cell):
        if cell.has_attr('rowspan'):
            rep_row = int(cell.attrs['rowspan'])
        else: # ~cell.has_attr('rowspan'):
            rep_row = 1
        if cell.has_attr('colspan'):
            rep_col = int(cell.attrs['colspan'])
        else: # ~cell.has_attr('colspan'):
            rep_col = 1 

        return (rep_row, rep_col)

# function to apply the data to all rows in span
def process_rows(rows, num_rows, num_cols):
    data = pd.DataFrame(np.ones((num_rows, num_cols))*np.nan)
    for i, row in enumerate(rows):
        try:
            col_stat = data.iloc[i,:][data.iloc[i,:].isnull()].index[0]
        except IndexError:
            print(i, row)

        for j, cell in enumerate(row.find_all(['td', 'th'])):
            rep_row, rep_col = get_spans(cell)

            #print("cols {0} to {1} with rep_col={2}".format(col_stat, col_stat+rep_col, rep_col))
            #print("\trows {0} to {1} with rep_row={2}".format(i, i+rep_row, rep_row))

            #find first non-na col and fill that one
            while any(data.iloc[i,col_stat:col_stat+rep_col].notnull()):
                col_stat+=1
            
            data.iloc[i:i+rep_row,col_stat:col_stat+rep_col] = cell.getText(strip = True) # strip will remove /n 
            if col_stat<data.shape[1]-1:
                col_stat+=rep_col
    return data

In [5]:
# call functions
rows, num_rows, num_cols = pre_process_table(grammy_edition_table)
# convert into df
grammy_edition_df = process_rows(rows, num_rows, num_cols)

In [6]:
grammy_edition_df.head(10)

,0,1,2,3,4,5,6
0,Edition,Date,Venue,Venue City,Host,Network,Viewers(in millions)
1,1st,"May 4, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills&New York City,Mort Sahl,NBC,—
2,2nd,"November 29, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills&New York City,Meredith Willson,NBC,—
3,3rd,"April 13, 1961",Various (includingBeverly Hilton Hotel),Beverly Hills&New York City,None,NBC,—
4,4th,"May 29, 1962",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles&New York City",None,NBC,—
5,5th,"May 15, 1963",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles&New York City",Frank Sinatra,NBC,—
6,6th,"May 12, 1964",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles&New York City",None,NBC,—
7,7th,"April 13, 1965",Beverly Hilton Hotel,Beverly Hills,None,NBC,—
8,8th,"March 15, 1966",Various,"Chicago,Los Angeles,NashvilleandNew York City",Jerry Lewis,NBC,—
9,9th,"March 2, 1967",Various,"Chicago,Los Angeles,NashvilleandNew York City",None,NBC,—


### Step 1
The first step will be to update the headers from the column numbers to the column headers

In [7]:
# what are the current column names
list(grammy_edition_df.columns)

[0, 1, 2, 3, 4, 5, 6]

In [8]:
# replace column names with first row from df
grammy_edition_df.columns = grammy_edition_df.iloc[0]

In [9]:
# remove first row so that there isn't a duplicate
grammy_edition_df = grammy_edition_df[1:]

In [10]:
list(grammy_edition_df.columns)

['Edition',
 'Date',
 'Venue',
 'Venue City',
 'Host',
 'Network',
 'Viewers(in millions)']

In [11]:
# The header Viewers(in millions) should have a space between the s and (
grammy_edition_df.rename(columns={'Viewers(in millions)':'Viewers (in millions)'}, inplace=True)

In [12]:
list(grammy_edition_df.columns)

['Edition',
 'Date',
 'Venue',
 'Venue City',
 'Host',
 'Network',
 'Viewers (in millions)']

### Step 2
The next step will be to add spacing on either side of ampersand using string replacement

In [13]:
grammy_edition_df.head(15)

,Edition,Date,Venue,Venue City,Host,Network,Viewers (in millions)
1,1st,"May 4, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills&New York City,Mort Sahl,NBC,—
2,2nd,"November 29, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills&New York City,Meredith Willson,NBC,—
3,3rd,"April 13, 1961",Various (includingBeverly Hilton Hotel),Beverly Hills&New York City,None,NBC,—
4,4th,"May 29, 1962",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles&New York City",None,NBC,—
5,5th,"May 15, 1963",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles&New York City",Frank Sinatra,NBC,—
6,6th,"May 12, 1964",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles&New York City",None,NBC,—
7,7th,"April 13, 1965",Beverly Hilton Hotel,Beverly Hills,None,NBC,—
8,8th,"March 15, 1966",Various,"Chicago,Los Angeles,NashvilleandNew York City",Jerry Lewis,NBC,—
9,9th,"March 2, 1967",Various,"Chicago,Los Angeles,NashvilleandNew York City",None,NBC,—
10,10th,"February 29, 1968",Various,"Chicago,Los Angeles,NashvilleandNew York City",None,NBC,—


In [14]:
grammy_edition_df['Venue City'] = grammy_edition_df['Venue City'].str.replace(r"&"," & ", regex=True)

In [15]:
grammy_edition_df.head(10)

,Edition,Date,Venue,Venue City,Host,Network,Viewers (in millions)
1,1st,"May 4, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,Mort Sahl,NBC,—
2,2nd,"November 29, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,Meredith Willson,NBC,—
3,3rd,"April 13, 1961",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,None,NBC,—
4,4th,"May 29, 1962",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles & New York City",None,NBC,—
5,5th,"May 15, 1963",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles & New York City",Frank Sinatra,NBC,—
6,6th,"May 12, 1964",Various (includingBeverly Hilton Hotel),"Chicago,Los Angeles & New York City",None,NBC,—
7,7th,"April 13, 1965",Beverly Hilton Hotel,Beverly Hills,None,NBC,—
8,8th,"March 15, 1966",Various,"Chicago,Los Angeles,NashvilleandNew York City",Jerry Lewis,NBC,—
9,9th,"March 2, 1967",Various,"Chicago,Los Angeles,NashvilleandNew York City",None,NBC,—
10,10th,"February 29, 1968",Various,"Chicago,Los Angeles,NashvilleandNew York City",None,NBC,—


### Step 3
Next will be adding spaces after commas using string replacement

In [16]:
grammy_edition_df['Venue City'] = grammy_edition_df['Venue City'].str.replace(r",",", ", regex=True)

In [17]:
grammy_edition_df.head(10)

,Edition,Date,Venue,Venue City,Host,Network,Viewers (in millions)
1,1st,"May 4, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,Mort Sahl,NBC,—
2,2nd,"November 29, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,Meredith Willson,NBC,—
3,3rd,"April 13, 1961",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,None,NBC,—
4,4th,"May 29, 1962",Various (includingBeverly Hilton Hotel),"Chicago, Los Angeles & New York City",None,NBC,—
5,5th,"May 15, 1963",Various (includingBeverly Hilton Hotel),"Chicago, Los Angeles & New York City",Frank Sinatra,NBC,—
6,6th,"May 12, 1964",Various (includingBeverly Hilton Hotel),"Chicago, Los Angeles & New York City",None,NBC,—
7,7th,"April 13, 1965",Beverly Hilton Hotel,Beverly Hills,None,NBC,—
8,8th,"March 15, 1966",Various,"Chicago, Los Angeles, NashvilleandNew York City",Jerry Lewis,NBC,—
9,9th,"March 2, 1967",Various,"Chicago, Los Angeles, NashvilleandNew York City",None,NBC,—
10,10th,"February 29, 1968",Various,"Chicago, Los Angeles, NashvilleandNew York City",None,NBC,—


### Step 4
Again updating the string of Venue city as NashvilleandNew York is difficult to read.  Additionally, there is are inconsistencies in '&' vs 'and' so making that consistent with an ampersand - using string replacement

In [18]:
grammy_edition_df['Venue City'] = grammy_edition_df['Venue City'].str.replace(r"eandN","e & N", regex=True)

In [19]:
grammy_edition_df.head(10)

,Edition,Date,Venue,Venue City,Host,Network,Viewers (in millions)
1,1st,"May 4, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,Mort Sahl,NBC,—
2,2nd,"November 29, 1959",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,Meredith Willson,NBC,—
3,3rd,"April 13, 1961",Various (includingBeverly Hilton Hotel),Beverly Hills & New York City,None,NBC,—
4,4th,"May 29, 1962",Various (includingBeverly Hilton Hotel),"Chicago, Los Angeles & New York City",None,NBC,—
5,5th,"May 15, 1963",Various (includingBeverly Hilton Hotel),"Chicago, Los Angeles & New York City",Frank Sinatra,NBC,—
6,6th,"May 12, 1964",Various (includingBeverly Hilton Hotel),"Chicago, Los Angeles & New York City",None,NBC,—
7,7th,"April 13, 1965",Beverly Hilton Hotel,Beverly Hills,None,NBC,—
8,8th,"March 15, 1966",Various,"Chicago, Los Angeles, Nashville & New York City",Jerry Lewis,NBC,—
9,9th,"March 2, 1967",Various,"Chicago, Los Angeles, Nashville & New York City",None,NBC,—
10,10th,"February 29, 1968",Various,"Chicago, Los Angeles, Nashville & New York City",None,NBC,—


### Step 5
I also need to add a space to the Venue column for the first few that don't have a space between including and Beverly using string replacement

In [20]:
grammy_edition_df.columns = grammy_edition_df.columns.str.replace('Viewers(in millions)', 'Viewers (in millions)', regex=True)


In [21]:
grammy_edition_df['Venue'] = grammy_edition_df['Venue'].str.replace(r"gB","g B", regex=True)

In [22]:
grammy_edition_df

,Edition,Date,Venue,Venue City,Host,Network,Viewers (in millions)
1,1st,"May 4, 1959",Various (including Beverly Hilton Hotel),Beverly Hills & New York City,Mort Sahl,NBC,—
2,2nd,"November 29, 1959",Various (including Beverly Hilton Hotel),Beverly Hills & New York City,Meredith Willson,NBC,—
3,3rd,"April 13, 1961",Various (including Beverly Hilton Hotel),Beverly Hills & New York City,None,NBC,—
4,4th,"May 29, 1962",Various (including Beverly Hilton Hotel),"Chicago, Los Angeles & New York City",None,NBC,—
5,5th,"May 15, 1963",Various (including Beverly Hilton Hotel),"Chicago, Los Angeles & New York City",Frank Sinatra,NBC,—
...,...,...,...,...,...,...,...
62,62nd,"January 26, 2020",Staples Center,Los Angeles,Alicia Keys,CBS,18.70[37]
63,63rd,"March 14, 2021",Los Angeles Convention Center,Los Angeles,Trevor Noah,CBS,9.23[38]
64,64th,"April 3, 2022",MGM Grand Garden Arena,Las Vegas,Trevor Noah,CBS,9.59[39]
65,65th,"February 5, 2023",Crypto.com Arena[40],Los Angeles,Trevor Noah,CBS,12.55[41]


### Step 6
Removing numbers in brackets that were links to other pages using string replacement

In [23]:
grammy_edition_df['Viewers (in millions)'] = grammy_edition_df['Viewers (in millions)'].str.replace(r"\[.*\]","", regex=True)
grammy_edition_df['Venue'] = grammy_edition_df['Venue'].str.replace(r"\[.*\]","", regex=True)

In [24]:
grammy_edition_df

,Edition,Date,Venue,Venue City,Host,Network,Viewers (in millions)
1,1st,"May 4, 1959",Various (including Beverly Hilton Hotel),Beverly Hills & New York City,Mort Sahl,NBC,—
2,2nd,"November 29, 1959",Various (including Beverly Hilton Hotel),Beverly Hills & New York City,Meredith Willson,NBC,—
3,3rd,"April 13, 1961",Various (including Beverly Hilton Hotel),Beverly Hills & New York City,None,NBC,—
4,4th,"May 29, 1962",Various (including Beverly Hilton Hotel),"Chicago, Los Angeles & New York City",None,NBC,—
5,5th,"May 15, 1963",Various (including Beverly Hilton Hotel),"Chicago, Los Angeles & New York City",Frank Sinatra,NBC,—
...,...,...,...,...,...,...,...
62,62nd,"January 26, 2020",Staples Center,Los Angeles,Alicia Keys,CBS,18.70
63,63rd,"March 14, 2021",Los Angeles Convention Center,Los Angeles,Trevor Noah,CBS,9.23
64,64th,"April 3, 2022",MGM Grand Garden Arena,Las Vegas,Trevor Noah,CBS,9.59
65,65th,"February 5, 2023",Crypto.com Arena,Los Angeles,Trevor Noah,CBS,12.55


In [25]:
grammy_edition_df['Date'] = grammy_edition_df['Date'].astype('datetime64[ns]')
grammy_edition_df['Date'] = grammy_edition_df['Date'].dt.to_period('Y')
grammy_edition_df = grammy_edition_df.rename(columns={'Date': 'Year'})
grammy_edition_df = grammy_edition_df.applymap(str)
grammy_edition_df.head()

,Edition,Year,Venue,Venue City,Host,Network,Viewers (in millions)
1,1st,1959,Various (including Beverly Hilton Hotel),Beverly Hills & New York City,Mort Sahl,NBC,—
2,2nd,1959,Various (including Beverly Hilton Hotel),Beverly Hills & New York City,Meredith Willson,NBC,—
3,3rd,1961,Various (including Beverly Hilton Hotel),Beverly Hills & New York City,None,NBC,—
4,4th,1962,Various (including Beverly Hilton Hotel),"Chicago, Los Angeles & New York City",None,NBC,—
5,5th,1963,Various (including Beverly Hilton Hotel),"Chicago, Los Angeles & New York City",Frank Sinatra,NBC,—


In [26]:
grammy_edition_df.to_csv('grammy_edition.csv', index=False) 